<a href="https://colab.research.google.com/github/beamalves/TCC1/blob/main/TCC1__etapa1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
"""
Script Final: Processamento, União e Análise de Capacidade com Georreferenciamento

Este script integra todas as etapas em um único fluxo de trabalho:
1. Processamento da planilha DataRecords_SUB_ONS_SINDAT
2. União com a planilha de Capacidade Remanescente
3. Análise e visualização dos dados combinados
4. Exportação de shapefile para QGIS

Modificações:
- A coluna 'Nome_Planilha SINDAT ONS' contém apenas nomes que incluem "sub" ou "SUB"
- Corrigido o erro de criação do shapefile (adicionada extensão .shp ao caminho)
"""

# Instalar todas as bibliotecas necessárias
!pip install pandas numpy matplotlib seaborn folium unidecode openpyxl geopandas -q

# Importar bibliotecas após instalação
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import unidecode
import folium
from folium.plugins import MarkerCluster, HeatMap
import os
import io
import time
from google.colab import files
import geopandas as gpd
from shapely.geometry import Point

# Configuração inicial
print("===== CONFIGURAÇÃO INICIAL =====")
print("Bibliotecas carregadas com sucesso!")

# Criar diretórios para resultados
if not os.path.exists('resultados'):
    os.makedirs('resultados')
if not os.path.exists('resultados/visualizacoes'):
    os.makedirs('resultados/visualizacoes')
if not os.path.exists('resultados/shapefile'):
    os.makedirs('resultados/shapefile')

# Configurações de visualização
plt.style.use('ggplot')
sns.set(style="whitegrid")

# Timer para medir tempo de execução
tempo_inicio = time.time()

# ===== ETAPA 1: PROCESSAMENTO DA PLANILHA DE COORDENADAS =====
print("\n===== ETAPA 1: PROCESSAMENTO DA PLANILHA DE COORDENADAS =====")

print("Faça upload do arquivo 'DataRecords_SUB_ONS_SINDAT_24abr2025.csv':")
uploaded_coords = files.upload()
filename_coords = list(uploaded_coords.keys())[0]

# Ler arquivo CSV
if filename_coords.endswith('.csv'):
    # Tentar primeiro abrir como CSV normal
    try:
        print("Tentando abrir como CSV com separador ';'...")
        df_coords = pd.read_csv(io.BytesIO(uploaded_coords[filename_coords]), sep=';', encoding='utf-8')

        # Verificar se há uma coluna com coordenadas x,y na última coluna
        ultima_coluna = df_coords.columns[-1]
        print(f"Última coluna: {ultima_coluna}")

        # Ver se a última coluna tem valores com vírgulas para x,y
        amostra = df_coords[ultima_coluna].iloc[0] if len(df_coords) > 0 else ""
        print(f"Amostra da última coluna: {amostra}")

        if ',' in str(amostra):
            print("Detectado formato de coordenadas na última coluna. Processando...")
            # Extrair coordenadas x e y
            coords = df_coords[ultima_coluna].str.split(',', expand=True)
            if len(coords.columns) >= 3:
                df_coords['x'] = pd.to_numeric(coords[1], errors='coerce')
                df_coords['y'] = pd.to_numeric(coords[2], errors='coerce')

        # Verificar se existe a coluna 'Nome'
        if 'Nome' in df_coords.columns:
            print("Coluna 'Nome' encontrada. Processando pontos de conexão...")
            # Guardar o nome original da subestação
            df_coords['Nome_Original'] = df_coords['Nome']

            # Processar nomes para pontos de conexão
            df_coords['Ponto de Conexão'] = df_coords['Nome'].apply(
                lambda x: re.sub(r'\bsub\b', '', str(x), flags=re.IGNORECASE).strip() if pd.notna(x) else ""
            )

            # Filtrar Nome_Original para manter apenas os que contêm "sub" ou "SUB"
            def filtrar_nome_sub(nome):
                if pd.isna(nome):
                    return None
                if 'sub' in nome.lower():
                    return nome
                return None

            df_coords['Nome_Original'] = df_coords['Nome_Original'].apply(filtrar_nome_sub)
            print("Filtrando nomes originais: mantendo apenas os que contêm 'sub' ou 'SUB'")

        else:
            print("ALERTA: Coluna 'Nome' não encontrada. Não será possível obter o nome original da subestação.")
            df_coords['Nome_Original'] = None

        # Criar coluna UF se não existir
        if 'UF' not in df_coords.columns:
            print("Coluna 'UF' não encontrada. Criando coluna vazia...")
            df_coords['UF'] = ""

            # Tentar extrair UF do nome
            def extract_uf(nome):
                # Padrões comuns de UF no final do nome
                uf_patterns = [
                    r'\(([A-Z]{2})\)$',  # UF entre parênteses no final: (SP)
                    r'\*([A-Z]{2})\*$',   # UF entre asteriscos no final: *SP*
                    r' ([A-Z]{2})$'       # UF como duas letras maiúsculas no final após espaço
                ]

                # Tentar cada padrão
                for pattern in uf_patterns:
                    match = re.search(pattern, nome)
                    if match:
                        uf = match.group(1)
                        return uf

                return ""

            # Aplicar extração de UF se tivermos coluna de nome
            if 'Ponto de Conexão' in df_coords.columns:
                df_coords['UF'] = df_coords['Ponto de Conexão'].apply(extract_uf)

    except Exception as e:
        # Se falhar, processar o arquivo manualmente
        print(f"Erro ao processar CSV normalmente: {e}")
        print("Processando o arquivo CSV manualmente...")
        raw_content = io.BytesIO(uploaded_coords[filename_coords]).read().decode('utf-8')
        lines = raw_content.splitlines()

        # Extrair cabeçalho
        header = lines[0].split(';')
        print(f"Cabeçalho original: {header}")

        # Criar listas para armazenar os dados processados
        pontos_conexao = []
        nomes_originais = []
        ufs = []
        x_coords = []
        y_coords = []

        # Função para extrair UF do nome
        def extract_uf(nome):
            # Padrões comuns de UF no final do nome
            uf_patterns = [
                r'\(([A-Z]{2})\)$',  # UF entre parênteses no final: (SP)
                r'\*([A-Z]{2})\*$',   # UF entre asteriscos no final: *SP*
                r' ([A-Z]{2})$'       # UF como duas letras maiúsculas no final após espaço
            ]

            # Tentar cada padrão
            for pattern in uf_patterns:
                match = re.search(pattern, nome)
                if match:
                    uf = match.group(1)
                    nome_sem_uf = re.sub(pattern, '', nome).strip()
                    return nome_sem_uf, uf

            # Se nenhum padrão foi encontrado, retornar nome original e UF vazia
            return nome, ""

        # Processar cada linha
        for i in range(1, len(lines)):
            if not lines[i].strip():
                continue  # Pular linhas vazias

            parts = lines[i].split(';')

            # Obter o nome (coluna "Nome" é a 3ª coluna, índice 2)
            nome = parts[2].strip() if len(parts) > 2 and parts[2] else ""

            # Guardar o nome original, apenas se contiver "sub" ou "SUB"
            nome_original = nome if nome and 'sub' in nome.lower() else None

            # Processar as coordenadas (última coluna contém ",x,y")
            coords_str = parts[-1].strip() if len(parts) > 0 else ""
            coords = coords_str.split(',')

            # Extrair x e y se disponíveis
            x_val = coords[1].strip() if len(coords) > 1 else ""
            y_val = coords[2].strip() if len(coords) > 2 else ""

            # Processar nome para extrair UF e remover "sub"
            ponto_conexao = nome
            if nome:
                # Remover a palavra "sub" do nome
                ponto_conexao = re.sub(r'\bsub\b', '', ponto_conexao, flags=re.IGNORECASE).strip()

                # Extrair UF do nome
                ponto_conexao, uf = extract_uf(ponto_conexao)

                # Remover espaços múltiplos
                ponto_conexao = re.sub(r'\s+', ' ', ponto_conexao).strip()
            else:
                uf = ""

            # Adicionar dados às listas
            pontos_conexao.append(ponto_conexao)
            nomes_originais.append(nome_original)
            ufs.append(uf)
            x_coords.append(x_val)
            y_coords.append(y_val)

        # Criar DataFrame com os dados processados
        df_coords = pd.DataFrame({
            'Ponto de Conexão': pontos_conexao,
            'Nome_Original': nomes_originais,
            'UF': ufs,
            'x': x_coords,
            'y': y_coords
        })

        # Converter coordenadas para tipo numérico
        df_coords['x'] = pd.to_numeric(df_coords['x'], errors='coerce')
        df_coords['y'] = pd.to_numeric(df_coords['y'], errors='coerce')
else:
    print(f"Formato de arquivo não suportado: {filename_coords}")
    raise ValueError("Por favor, forneça um arquivo CSV.")

# Adicionar região com base na UF
regioes_por_uf = {
    'AC': 'Norte', 'AM': 'Norte', 'AP': 'Norte', 'PA': 'Norte',
    'RO': 'Norte', 'RR': 'Norte', 'TO': 'Norte',
    'AL': 'Nordeste', 'BA': 'Nordeste', 'CE': 'Nordeste', 'MA': 'Nordeste',
    'PB': 'Nordeste', 'PE': 'Nordeste', 'PI': 'Nordeste', 'RN': 'Nordeste', 'SE': 'Nordeste',
    'DF': 'Centro-Oeste', 'GO': 'Centro-Oeste', 'MS': 'Centro-Oeste', 'MT': 'Centro-Oeste',
    'ES': 'Sudeste', 'MG': 'Sudeste', 'RJ': 'Sudeste', 'SP': 'Sudeste',
    'PR': 'Sul', 'RS': 'Sul', 'SC': 'Sul'
}
df_coords['Regiao'] = df_coords['UF'].map(regioes_por_uf)

# Verificar coordenadas válidas
# Brasil: aproximadamente entre -73.99 e -34.79 de longitude (x) e -33.75 e 5.27 de latitude (y)
df_coords['Coordenadas_Validas'] = (
    (df_coords['x'] >= -73.99) & (df_coords['x'] <= -34.79) &
    (df_coords['y'] >= -33.75) & (df_coords['y'] <= 5.27)
)

print("\nProcessamento da planilha de coordenadas concluído!")
print(f"Total de registros processados: {len(df_coords)}")
print(f"Registros com coordenadas válidas: {df_coords['Coordenadas_Validas'].sum()} de {len(df_coords)}")
print(f"Registros com UF identificada: {(df_coords['UF'] != '').sum()} de {len(df_coords)}")
print(f"Registros com nome original contendo 'sub': {df_coords['Nome_Original'].notna().sum()} de {len(df_coords)}")

# Salvar o resultado intermediário
df_coords.to_excel('resultados/DataRecords_SUB_ONS_SINDAT_processado.xlsx', index=False)
print("Arquivo intermediário 'DataRecords_SUB_ONS_SINDAT_processado.xlsx' gerado na pasta 'resultados'.")

# Mostrar amostra dos dados processados
print("\nAmostra dos dados processados:")
print(df_coords.head())

# ===== ETAPA 2: UNIÃO DAS PLANILHAS =====
print("\n===== ETAPA 2: UNIÃO DAS PLANILHAS =====")

# Upload do arquivo de capacidade remanescente
print("\nFaça upload do arquivo de capacidade remanescente (Capacidade_Remanescente_SIN_para_Escoamento_de_Geração-1_2025_24abr2025.xlsx):")
uploaded_cap = files.upload()
filename_cap = list(uploaded_cap.keys())[0]

# Ler arquivo de capacidade
if filename_cap.endswith('.xlsx'):
    df_cap = pd.read_excel(io.BytesIO(uploaded_cap[filename_cap]))
else:
    print(f"Formato de arquivo não suportado: {filename_cap}")
    raise ValueError("Por favor, forneça um arquivo Excel (.xlsx).")

# Verificar estruturas dos DataFrames
print("\nEstrutura da planilha de capacidade:")
print(f"Número de registros: {len(df_cap)}")
print("Colunas disponíveis:")
print(df_cap.columns.tolist())

# Função para normalizar texto para correspondência
def normalizar_texto(texto):
    if pd.isna(texto) or texto == '':
        return ''
    # Converter para string caso não seja
    texto = str(texto)
    # Converter para minúsculas
    texto = texto.lower()
    # Remover acentos
    texto = unidecode.unidecode(texto)
    # Remover palavras como "sub", "subestacao", etc.
    texto = re.sub(r'\bsub(estacao|estação)?\b', '', texto, flags=re.IGNORECASE)
    texto = re.sub(r'\bse\b', '', texto, flags=re.IGNORECASE)
    # Remover caracteres especiais e números
    texto = re.sub(r'[^a-z ]', ' ', texto)
    # Remover espaços extras
    texto = re.sub(r'\s+', ' ', texto).strip()
    return texto

# Identificar coluna de ponto de conexão na planilha de capacidade
ponto_conexao_cap = None
for col in df_cap.columns:
    if 'ponto' in col.lower() and 'conex' in col.lower():
        ponto_conexao_cap = col
        break

if not ponto_conexao_cap:
    # Tentar encontrar uma coluna que pode ter os pontos de conexão
    possibilidades = ['Subestação', 'SE', 'Instalação', 'Nome', 'Barramento']
    for col in possibilidades:
        if col in df_cap.columns:
            ponto_conexao_cap = col
            break

print(f"\nColuna de ponto de conexão na planilha de capacidade: {ponto_conexao_cap}")

# Verificar se a coluna Ponto de Conexão existe em df_coords
if 'Ponto de Conexão' not in df_coords.columns:
    print("ERRO: Coluna 'Ponto de Conexão' não encontrada na planilha de coordenadas!")
    # Tentar criar a partir de alguma outra coluna
    if 'Nome' in df_coords.columns:
        print("Criando 'Ponto de Conexão' a partir da coluna 'Nome'...")
        df_coords['Ponto de Conexão'] = df_coords['Nome'].apply(
            lambda x: re.sub(r'\bsub\b', '', str(x), flags=re.IGNORECASE).strip() if pd.notna(x) else ""
        )
    else:
        # Se não houver coluna adequada, usar o próprio índice
        print("Nenhuma coluna de nome encontrada. Usando valores de índice para pontos de conexão...")
        df_coords['Ponto de Conexão'] = [f"Ponto {i+1}" for i in range(len(df_coords))]

# Adicionar colunas normalizadas para correspondência
df_coords['ponto_normalizado'] = df_coords['Ponto de Conexão'].apply(normalizar_texto)
df_cap['ponto_normalizado'] = df_cap[ponto_conexao_cap].apply(normalizar_texto)

# Comparar os pontos de conexão
pontos_coords = set(df_coords['ponto_normalizado'].dropna())
pontos_cap = set(df_cap['ponto_normalizado'].dropna())

# Pontos em comum
pontos_comuns = pontos_coords.intersection(pontos_cap)
print(f"\nNúmero de pontos de conexão em comum: {len(pontos_comuns)}")

# Pontos na planilha de capacidade que não estão na planilha de coordenadas
pontos_faltantes = pontos_cap - pontos_coords
print(f"Número de pontos na planilha de capacidade sem georreferenciamento: {len(pontos_faltantes)}")

# Função para encontrar a melhor correspondência usando similaridade
def encontrar_melhor_correspondencia(nome, lista_nomes):
    # Para nomes muito curtos, exigir correspondência exata
    if len(nome) <= 3:
        for n in lista_nomes:
            if nome == n:
                return n
        return None

    # Para nomes mais longos, usar similaridade
    melhor_score = 0
    melhor_match = None

    for n in lista_nomes:
        # Calcular similaridade como proporção de palavras em comum
        palavras_nome = set(nome.split())
        palavras_n = set(n.split())

        if not palavras_nome or not palavras_n:
            continue

        palavras_comuns = palavras_nome.intersection(palavras_n)

        # Calcular score como proporção de palavras em comum
        score_nome = len(palavras_comuns) / len(palavras_nome) if len(palavras_nome) > 0 else 0
        score_n = len(palavras_comuns) / len(palavras_n) if len(palavras_n) > 0 else 0
        score = min(score_nome, score_n)  # Usar o menor score

        if score > melhor_score and score >= 0.5:  # Exigir pelo menos 50% de similaridade
            melhor_score = score
            melhor_match = n

    return melhor_match

# Criar dicionário de correspondência para pontos não encontrados diretamente
correspondencia_manual = {}
lista_pontos_coords = list(pontos_coords)

for ponto_cap in pontos_faltantes:
    melhor_match = encontrar_melhor_correspondencia(ponto_cap, lista_pontos_coords)
    if melhor_match:
        correspondencia_manual[ponto_cap] = melhor_match

print(f"Encontradas {len(correspondencia_manual)} correspondências aproximadas para pontos faltantes")

# Realizar a junção das tabelas (correspondência direta)
df_resultado = pd.merge(
    df_cap,
    df_coords[['ponto_normalizado', 'x', 'y', 'UF', 'Regiao', 'Coordenadas_Validas', 'Nome_Original']],
    on='ponto_normalizado',
    how='left'
)

# Aplicar correspondência manual para pontos faltantes
registros_atualizados = 0
for idx, row in df_resultado.iterrows():
    if pd.isna(row['x']) and row['ponto_normalizado'] in correspondencia_manual:
        ponto_correspondente = correspondencia_manual[row['ponto_normalizado']]
        dados_coords = df_coords[df_coords['ponto_normalizado'] == ponto_correspondente]

        if not dados_coords.empty:
            df_resultado.loc[idx, 'x'] = dados_coords['x'].values[0]
            df_resultado.loc[idx, 'y'] = dados_coords['y'].values[0]

            if 'UF' in dados_coords.columns:
                df_resultado.loc[idx, 'UF'] = dados_coords['UF'].values[0]

            if 'Regiao' in dados_coords.columns:
                df_resultado.loc[idx, 'Regiao'] = dados_coords['Regiao'].values[0]

            if 'Coordenadas_Validas' in dados_coords.columns:
                df_resultado.loc[idx, 'Coordenadas_Validas'] = dados_coords['Coordenadas_Validas'].values[0]

            if 'Nome_Original' in dados_coords.columns:
                df_resultado.loc[idx, 'Nome_Original'] = dados_coords['Nome_Original'].values[0]

            registros_atualizados += 1

print(f"Atualizados {registros_atualizados} registros adicionais com correspondência aproximada")

# Verificar se há colunas duplicadas
colunas_duplicadas = df_resultado.columns[df_resultado.columns.duplicated()]
if len(colunas_duplicadas) > 0:
    print(f"\nATENÇÃO: Foram encontradas colunas duplicadas: {list(colunas_duplicadas)}")
    # Remover colunas duplicadas
    df_resultado = df_resultado.loc[:, ~df_resultado.columns.duplicated()]
    print("Colunas duplicadas foram removidas.")

# Identificar e mapear as colunas necessárias para o formato final
colunas_mapeadas = {
    'Região': 'Regiao',
    'UF': 'UF',
    'Ponto de Conexão': ponto_conexao_cap,
    'Tensão (kV)': None,
    'N° Barra': None,
    'PA (MW)': None,
    'CUST (MW)': None,
    'Ano': None,
    'Margem (MW)': None,
    'FatorLimitante': None,
    'Observacao': None,
    'Nome_Planilha SINDAT ONS': 'Nome_Original',  # Usar o nome original da subestação
    'Coordenada X': 'x',
    'Coordenada Y': 'y'
}

# Buscar correspondências automáticas para colunas não mapeadas
for col_destino in colunas_mapeadas:
    if colunas_mapeadas[col_destino] is None:
        # Procurar colunas com nomes similares
        for col_original in df_resultado.columns:
            # Normalizar os nomes para comparação
            col_dest_norm = normalizar_texto(col_destino)
            col_orig_norm = normalizar_texto(col_original)

            # Verificar se há palavras-chave em comum
            palavras_dest = set(col_dest_norm.split())
            palavras_orig = set(col_orig_norm.split())
            palavras_comuns = palavras_dest.intersection(palavras_orig)

            # Se houver pelo menos uma palavra em comum e for um bom candidato
            if len(palavras_comuns) > 0 and (
                col_dest_norm in col_orig_norm or
                len(palavras_comuns) / len(palavras_dest) >= 0.5
            ):
                colunas_mapeadas[col_destino] = col_original
                break

# Mostrar o mapeamento encontrado
print("\nMapeamento de colunas:")
for col_destino, col_original in colunas_mapeadas.items():
    print(f"{col_destino} -> {col_original}")

# Criar o DataFrame final
cols_final = [
    'Região', 'UF', 'Ponto de Conexão', 'Tensão (kV)', 'N° Barra',
    'PA (MW)', 'CUST (MW)', 'Ano', 'Margem (MW)', 'FatorLimitante',
    'Observacao', 'Nome_Planilha SINDAT ONS', 'Coordenada X', 'Coordenada Y'
]

df_final = pd.DataFrame(index=df_resultado.index)

# Preencher o DataFrame final
for col_destino in cols_final:
    col_original = colunas_mapeadas[col_destino]

    if col_original and col_original in df_resultado.columns:
        df_final[col_destino] = df_resultado[col_original]
    else:
        df_final[col_destino] = None  # Criar coluna vazia

# Caso a coluna 'Região' não tenha sido preenchida mas temos 'UF'
if df_final['Região'].isna().all() and not df_final['UF'].isna().all():
    df_final['Região'] = df_final['UF'].map(regioes_por_uf)

# Para a coluna Nome_Planilha SINDAT ONS, manter apenas nomes com "sub" ou "SUB"
nome_col = 'Nome_Planilha SINDAT ONS'
if nome_col in df_final.columns:
    # Verificar quais valores não contêm "sub" ou "SUB"
    sem_sub = df_final[nome_col].apply(
        lambda x: False if pd.isna(x) else ('sub' not in str(x).lower())
    )

    # Definir esses valores como nulos
    df_final.loc[sem_sub, nome_col] = None

    print(f"\nFiltro aplicado na coluna '{nome_col}': mantidos apenas nomes com 'sub' ou 'SUB'")
    print(f"Valores nulos após filtro: {df_final[nome_col].isna().sum()} de {len(df_final)}")

# Estatísticas finais
print("\nEstatísticas finais:")
print(f"Total de registros: {len(df_final)}")
print(f"Registros com coordenadas: {df_final['Coordenada X'].notna().sum()} ({df_final['Coordenada X'].notna().sum()/len(df_final)*100:.1f}%)")
print(f"Registros sem coordenadas: {df_final['Coordenada X'].isna().sum()} ({df_final['Coordenada X'].isna().sum()/len(df_final)*100:.1f}%)")
print(f"Registros com nome original de subestação: {df_final['Nome_Planilha SINDAT ONS'].notna().sum()} ({df_final['Nome_Planilha SINDAT ONS'].notna().sum()/len(df_final)*100:.1f}%)")

# Salvar o DataFrame final
output_filename = 'resultados/Capacidade_Remanescente_com_Georreferenciamento.xlsx'
df_final.to_excel(output_filename, index=False)
print(f"\nPlanilha final gerada: {output_filename}")

# ===== ETAPA 3: ANÁLISE E VISUALIZAÇÕES =====
print("\n===== ETAPA 3: ANÁLISE E VISUALIZAÇÕES =====")

# Converter colunas numéricas
colunas_potencia = ['PA (MW)', 'CUST (MW)', 'Margem (MW)', 'Tensão (kV)']
for col in colunas_potencia:
    if col in df_final.columns:
        df_final[col] = pd.to_numeric(df_final[col], errors='coerce')

# Criar categorias para análise
if 'Margem (MW)' in df_final.columns and not df_final['Margem (MW)'].isna().all():
    # Criar categorias de margem
    bins = [-np.inf, 0, 100, 300, 500, np.inf]
    labels = ['Negativa', 'Baixa (0-100)', 'Média (100-300)', 'Alta (300-500)', 'Muito Alta (>500)']
    df_final['Categoria Margem'] = pd.cut(df_final['Margem (MW)'], bins=bins, labels=labels)

    # Substituir valores NaN por "Sem Dados"
    df_final['Categoria Margem'] = df_final['Categoria Margem'].cat.add_categories(['Sem Dados'])
    df_final['Categoria Margem'] = df_final['Categoria Margem'].fillna('Sem Dados')

if 'Tensão (kV)' in df_final.columns and not df_final['Tensão (kV)'].isna().all():
    # Criar categorias de tensão
    df_final['Categoria Tensão'] = 'Sem Dados'

    # Categorias comuns de tensão no sistema elétrico brasileiro
    df_final.loc[df_final['Tensão (kV)'] <= 69, 'Categoria Tensão'] = 'Baixa (≤69 kV)'
    df_final.loc[(df_final['Tensão (kV)'] > 69) & (df_final['Tensão (kV)'] <= 138), 'Categoria Tensão'] = 'Média (69-138 kV)'
    df_final.loc[(df_final['Tensão (kV)'] > 138) & (df_final['Tensão (kV)'] <= 230), 'Categoria Tensão'] = 'Alta (138-230 kV)'
    df_final.loc[(df_final['Tensão (kV)'] > 230) & (df_final['Tensão (kV)'] <= 500), 'Categoria Tensão'] = 'Muito Alta (230-500 kV)'
    df_final.loc[df_final['Tensão (kV)'] > 500, 'Categoria Tensão'] = 'Extra Alta (>500 kV)'

# Criar análises básicas
print("Gerando visualizações básicas...")

# Mapa com pontos de conexão por categoria de margem
df_mapa = df_final.dropna(subset=['Coordenada X', 'Coordenada Y'])
if len(df_mapa) > 0:
    mapa = folium.Map(location=[-15.77972, -47.92972], zoom_start=5)  # Centro do Brasil

    # Criar uma função para determinar a cor com base na margem
    def get_color(margem):
        if pd.isna(margem):
            return 'gray'
        elif margem < 0:
            return 'red'
        elif margem < 100:
            return 'orange'
        elif margem < 300:
            return 'blue'
        elif margem < 500:
            return 'green'
        else:
            return 'purple'

    # Adicionar legenda ao mapa
    legend_html = """
    <div style="position: fixed;
                bottom: 50px; left: 50px; width: 180px; height: 160px;
                border:2px solid grey; z-index:9999; font-size:12px;
                background-color:white;
                padding: 10px;
                border-radius: 5px;">
    <b>Legenda (Margem MW)</b><br>
    <i class="fa fa-circle" style="color:red"></i> Negativa<br>
    <i class="fa fa-circle" style="color:orange"></i> Baixa (0-100)<br>
    <i class="fa fa-circle" style="color:blue"></i> Média (100-300)<br>
    <i class="fa fa-circle" style="color:green"></i> Alta (300-500)<br>
    <i class="fa fa-circle" style="color:purple"></i> Muito Alta (>500)<br>
    <i class="fa fa-circle" style="color:gray"></i> Sem Dados<br>
    </div>
    """
    mapa.get_root().html.add_child(folium.Element(legend_html))

    # Adicionar marcadores ao mapa
    for _, row in df_mapa.iterrows():
        cor = get_color(row['Margem (MW)'] if 'Margem (MW)' in row and pd.notna(row['Margem (MW)']) else None)

        info = f"<b>{row['Ponto de Conexão']}</b><br>"
        if 'Nome_Planilha SINDAT ONS' in row and pd.notna(row['Nome_Planilha SINDAT ONS']):
            info += f"<b>Nome Original:</b> {row['Nome_Planilha SINDAT ONS']}<br>"

        for col in ['Região', 'UF', 'Tensão (kV)', 'PA (MW)', 'CUST (MW)',
                    'Ano', 'Margem (MW)', 'FatorLimitante']:
            if col in df_final.columns and pd.notna(row[col]):
                info += f"{col}: {row[col]}<br>"

        folium.CircleMarker(
            location=[row['Coordenada Y'], row['Coordenada X']],
            radius=8,
            popup=info,
            tooltip=row['Ponto de Conexão'],
            color=cor,
            fill=True,
            fill_color=cor
        ).add_to(mapa)

    # Salvar o mapa
    mapa.save('resultados/visualizacoes/mapa_capacidade_pontos.html')
    print("Mapa interativo gerado com sucesso!")
else:
    print("Não há pontos com coordenadas válidas para gerar o mapa.")

# Análise por região (se houver dados suficientes)
if 'Região' in df_final.columns and not df_final['Região'].isna().all():
    regioes_unicas = df_final['Região'].dropna().unique()
    if len(regioes_unicas) > 0:
        # Contagem de pontos por região
        plt.figure(figsize=(10, 6))
        sns.countplot(data=df_final, x='Região', order=df_final['Região'].value_counts().index)
        plt.title('Número de Pontos de Conexão por Região')
        plt.ylabel('Número de Pontos')
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.savefig('resultados/visualizacoes/contagem_por_regiao.png')
        plt.close()

        # Análise da margem por região (se disponível)
        if 'Margem (MW)' in df_final.columns and not df_final['Margem (MW)'].isna().all():
            # Estatísticas de margem por região
            margem_regiao = df_final.groupby('Região')['Margem (MW)'].agg(['mean', 'min', 'max', 'count']).reset_index()

            # Gráfico de barras para média de margem por região
            plt.figure(figsize=(10, 6))
            sns.barplot(data=margem_regiao, x='Região', y='mean')
            plt.title('Margem Média (MW) por Região')
            plt.ylabel('Margem Média (MW)')
            plt.xticks(rotation=45)
            plt.tight_layout()
            plt.savefig('resultados/visualizacoes/margem_media_por_regiao.png')
            plt.close()

# Análise por UF (se houver dados suficientes)
if 'UF' in df_final.columns and not df_final['UF'].isna().all():
    ufs_unicas = df_final['UF'].dropna().unique()
    if len(ufs_unicas) > 0:
        # Contagem de pontos por UF
        contagem_uf = df_final['UF'].value_counts().head(10)  # Top 10 UFs

        if len(contagem_uf) > 0:
            # Gráfico de barras para contagem por UF (top 10)
            plt.figure(figsize=(12, 6))
            sns.barplot(x=contagem_uf.index, y=contagem_uf.values)
            plt.title('Top 10 UFs com Mais Pontos de Conexão')
            plt.ylabel('Número de Pontos')
            plt.xlabel('UF')
            plt.xticks(rotation=45)
            plt.tight_layout()
            plt.savefig('resultados/visualizacoes/contagem_por_uf.png')
            plt.close()

# Análise por tensão (se disponível)
if 'Tensão (kV)' in df_final.columns and not df_final['Tensão (kV)'].isna().all():
    # Verificar se há valores válidos
    valores_validos = df_final['Tensão (kV)'].dropna()
    if len(valores_validos) > 0:
        # Histograma de tensão
        plt.figure(figsize=(10, 6))
        sns.histplot(valores_validos, bins=10, kde=True)
        plt.title('Distribuição de Tensão (kV)')
        plt.xlabel('Tensão (kV)')
        plt.ylabel('Frequência')
        plt.tight_layout()
        plt.savefig('resultados/visualizacoes/distribuicao_tensao.png')
        plt.close()

        # Gráfico de barras para contagem por categoria de tensão
        if 'Categoria Tensão' in df_final.columns:
            categorias_tensao = df_final['Categoria Tensão'].value_counts()
            if len(categorias_tensao) > 0:
                plt.figure(figsize=(10, 6))
                sns.countplot(data=df_final, x='Categoria Tensão', order=categorias_tensao.index)
                plt.title('Número de Pontos por Categoria de Tensão')
                plt.xlabel('Categoria de Tensão')
                plt.ylabel('Número de Pontos')
                plt.xticks(rotation=45)
                plt.tight_layout()
                plt.savefig('resultados/visualizacoes/contagem_por_categoria_tensao.png')
                plt.close()

# Análise de margem (se disponível)
if 'Margem (MW)' in df_final.columns and not df_final['Margem (MW)'].isna().all():
    # Verificar se há valores válidos
    valores_validos = df_final['Margem (MW)'].dropna()
    if len(valores_validos) > 0:
        # Histograma de margem
        plt.figure(figsize=(10, 6))
        sns.histplot(valores_validos, bins=20, kde=True)
        plt.title('Distribuição de Margem (MW)')
        plt.xlabel('Margem (MW)')
        plt.ylabel('Frequência')
        plt.tight_layout()
        plt.savefig('resultados/visualizacoes/distribuicao_margem.png')
        plt.close()

        # Contagem por categoria de margem
        if 'Categoria Margem' in df_final.columns:
            categorias_margem = df_final['Categoria Margem'].value_counts()
            if len(categorias_margem) > 0:
                plt.figure(figsize=(10, 6))
                sns.countplot(data=df_final, x='Categoria Margem', order=categorias_margem.index)
                plt.title('Número de Pontos por Categoria de Margem')
                plt.xlabel('Categoria de Margem')
                plt.ylabel('Número de Pontos')
                plt.xticks(rotation=45)
                plt.tight_layout()
                plt.savefig('resultados/visualizacoes/contagem_por_categoria_margem.png')
                plt.close()

# Exportar estatísticas para Excel
print("\nExportando estatísticas para Excel...")
with pd.ExcelWriter('resultados/Estatisticas_Capacidade_Remanescente.xlsx') as writer:
    # Dados gerais
    pd.DataFrame({
        'Métrica': ['Total de Registros', 'Registros com Coordenadas', 'Registros sem Coordenadas',
                   'Registros com Nome SUB', 'Margem Média (MW)', 'Margem Mínima (MW)', 'Margem Máxima (MW)'],
        'Valor': [len(df_final), df_final['Coordenada X'].notna().sum(), df_final['Coordenada X'].isna().sum(),
                 df_final['Nome_Planilha SINDAT ONS'].notna().sum(),
                 df_final['Margem (MW)'].mean() if 'Margem (MW)' in df_final.columns and not df_final['Margem (MW)'].isna().all() else None,
                 df_final['Margem (MW)'].min() if 'Margem (MW)' in df_final.columns and not df_final['Margem (MW)'].isna().all() else None,
                 df_final['Margem (MW)'].max() if 'Margem (MW)' in df_final.columns and not df_final['Margem (MW)'].isna().all() else None]
    }).to_excel(writer, sheet_name='Métricas Gerais', index=False)

    # Dados por região
    if 'Região' in df_final.columns and not df_final['Região'].isna().all():
        # Contagem por região
        df_final.groupby('Região').size().reset_index(name='Contagem').to_excel(writer, sheet_name='Contagem por Região', index=False)

        # Estatísticas de margem por região
        if 'Margem (MW)' in df_final.columns and not df_final['Margem (MW)'].isna().all():
            df_final.groupby('Região')['Margem (MW)'].agg(['count', 'mean', 'min', 'max']).reset_index().to_excel(
                writer, sheet_name='Margem por Região', index=False)

    # Dados por UF
    if 'UF' in df_final.columns and not df_final['UF'].isna().all():
        # Contagem por UF
        df_final.groupby('UF').size().reset_index(name='Contagem').sort_values('Contagem', ascending=False).to_excel(
            writer, sheet_name='Contagem por UF', index=False)

        # Estatísticas de margem por UF
        if 'Margem (MW)' in df_final.columns and not df_final['Margem (MW)'].isna().all():
            df_final.groupby('UF')['Margem (MW)'].agg(['count', 'mean', 'min', 'max']).reset_index().sort_values('count', ascending=False).to_excel(
                writer, sheet_name='Margem por UF', index=False)

    # Dados por categoria de tensão
    if 'Categoria Tensão' in df_final.columns:
        # Contagem por categoria de tensão
        df_final.groupby('Categoria Tensão').size().reset_index(name='Contagem').to_excel(
            writer, sheet_name='Contagem por Tensão', index=False)

        # Estatísticas de margem por categoria de tensão
        if 'Margem (MW)' in df_final.columns and not df_final['Margem (MW)'].isna().all():
            df_final.groupby('Categoria Tensão')['Margem (MW)'].agg(['count', 'mean', 'min', 'max']).reset_index().to_excel(
                writer, sheet_name='Margem por Tensão', index=False)

    # Dados por categoria de margem
    if 'Categoria Margem' in df_final.columns:
        # Contagem por categoria de margem
        df_final.groupby('Categoria Margem').size().reset_index(name='Contagem').to_excel(
            writer, sheet_name='Contagem por Cat Margem', index=False)

    # Dados por fator limitante
    if 'FatorLimitante' in df_final.columns and not df_final['FatorLimitante'].isna().all():
        # Contagem por fator limitante
        df_final.groupby('FatorLimitante').size().reset_index(name='Contagem').sort_values('Contagem', ascending=False).head(20).to_excel(
            writer, sheet_name='Contagem por Fator', index=False)

# ===== ETAPA 4: EXPORTAR SHAPEFILE PARA QGIS =====
print("\n===== ETAPA 4: EXPORTAR SHAPEFILE PARA QGIS =====")

# Filtrar para pontos com coordenadas válidas
df_shp = df_final.dropna(subset=['Coordenada X', 'Coordenada Y']).copy()

if len(df_shp) > 0:
    print(f"Criando shapefile com {len(df_shp)} pontos...")

    # Verificar se há colunas duplicadas antes de criar o GeoDataFrame
    if df_shp.columns.duplicated().any():
        colunas_duplicadas = df_shp.columns[df_shp.columns.duplicated()]
        print(f"Encontradas colunas duplicadas no dataframe: {list(colunas_duplicadas)}")
        print("Removendo colunas duplicadas...")
        df_shp = df_shp.loc[:, ~df_shp.columns.duplicated()]

    # Criar coluna de geometria para os pontos
    geometry = [Point(xy) for xy in zip(df_shp['Coordenada X'], df_shp['Coordenada Y'])]

    # Criar GeoDataFrame com a geometria e dados
    gdf = gpd.GeoDataFrame(df_shp, geometry=geometry, crs="EPSG:4326")

    # Renomear colunas longas para evitar truncamento em shapefile
    colunas_renomeadas = {}
    for col in gdf.columns:
        if len(col) > 10 and col != 'geometry':
            novo_nome = col[:10].replace(' ', '_')
            # Garantir que o novo nome não duplica nomes existentes
            contador = 1
            temp_nome = novo_nome
            while temp_nome in colunas_renomeadas.values():
                temp_nome = f"{novo_nome[:8]}_{contador}"
                contador += 1
            colunas_renomeadas[col] = temp_nome

    if colunas_renomeadas:
        print("Renomeando colunas para compatibilidade com shapefile:")
        for col_original, col_novo in colunas_renomeadas.items():
            print(f"  {col_original} -> {col_novo}")
        gdf = gdf.rename(columns=colunas_renomeadas)

    # Caminho para salvar os arquivos shapefile
    shapefile_dir = 'resultados/shapefile'
    shapefile_path = os.path.join(shapefile_dir, 'pontos_capacidade.shp')  # Adicionada extensão .shp

    # Salvar como shapefile
    gdf.to_file(shapefile_path, driver='ESRI Shapefile', encoding='utf-8')

    print(f"Shapefile criado com sucesso em: {shapefile_path}")

    # Compactar pasta de shapefile para download
    !zip -r resultados/shapefile_pontos_capacidade.zip resultados/shapefile/

    print("Shapefile compactado para download.")
else:
    print("Não há pontos com coordenadas válidas para criar o shapefile.")

# Compactar os resultados para download
print("\nCompactando todos os resultados para download...")
!zip -r resultados_capacidade_remanescente.zip resultados/

# Tempo de execução
tempo_total = time.time() - tempo_inicio
print(f"\nProcessamento concluído em {tempo_total:.2f} segundos!")
print("\nArquivos gerados:")
print("- resultados/DataRecords_SUB_ONS_SINDAT_processado.xlsx")
print("- resultados/Capacidade_Remanescente_com_Georreferenciamento.xlsx")
print("- resultados/Estatisticas_Capacidade_Remanescente.xlsx")
print("- resultados/visualizacoes/ (contém gráficos e mapas)")
print("- resultados/shapefile/ (contém arquivos shapefile para QGIS)")
print("- resultados/shapefile_pontos_capacidade.zip (shapefile compactado)")
print("- resultados_capacidade_remanescente.zip (todos os resultados compactados)")

# Fazer download dos principais arquivos
files.download('resultados/Capacidade_Remanescente_com_Georreferenciamento.xlsx')
files.download('resultados/Estatisticas_Capacidade_Remanescente.xlsx')
files.download('resultados/shapefile_pontos_capacidade.zip')
files.download('resultados_capacidade_remanescente.zip')

print("\nProcessamento completo! Obrigado por utilizar o script.")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.8/235.8 kB 4.6 MB/s eta 0:00:00
===== CONFIGURAÇÃO INICIAL =====
Bibliotecas carregadas com sucesso!

===== ETAPA 1: PROCESSAMENTO DA PLANILHA DE COORDENADAS =====
Faça upload do arquivo 'DataRecords_SUB_ONS_SINDAT_24abr2025.csv':


Saving DataRecords_SUB_ONS_SINDAT_24abr2025.csv to DataRecords_SUB_ONS_SINDAT_24abr2025.csv
Tentando abrir como CSV com separador ';'...
Última coluna: Data Entrada,x,y
Amostra da última coluna: ,-50.283333003999985,-29.95833300099997
Detectado formato de coordenadas na última coluna. Processando...
Coluna 'Nome' encontrada. Processando pontos de conexão...
Filtrando nomes originais: mantendo apenas os que contêm 'sub' ou 'SUB'
Coluna 'UF' não encontrada. Criando coluna vazia...

Processamento da planilha de coordenadas concluído!
Total de registros processados: 1267
Registros com coordenadas válidas: 1267 de 1267
Registros com UF identificada: 1240 de 1267
Registros com nome original contendo 'sub': 1227 de 1267
Arquivo intermediário 'DataRecords_SUB_ONS_SINDAT_processado.xlsx' gerado na pasta 'resultados'.

Amostra dos dados processados:
   OBJECTID Id da Instalação Nome Id Agente Principal Agente Principal  \
0         1              NaN  NaN                 NaN              NaN   


Saving Capacidade_Remanescente_SIN_para_Escoamento_de_Geração-1_2025_24abr2025.xlsx to Capacidade_Remanescente_SIN_para_Escoamento_de_Geração-1_2025_24abr2025.xlsx

Estrutura da planilha de capacidade:
Número de registros: 3197
Colunas disponíveis:
['Região', 'UF', 'Ponto de Conexão', 'Tensão (kV)', 'N° Barra', 'PA (MW)', 'CUST (MW)', 'Ano', 'Margem (MW)', 'FatorLimitante', 'Observacao']

Coluna de ponto de conexão na planilha de capacidade: Ponto de Conexão

Número de pontos de conexão em comum: 5
Número de pontos na planilha de capacidade sem georreferenciamento: 509
Encontradas 475 correspondências aproximadas para pontos faltantes
Atualizados 3005 registros adicionais com correspondência aproximada

Mapeamento de colunas:
Região -> Regiao
UF -> UF
Ponto de Conexão -> Ponto de Conexão
Tensão (kV) -> Tensão (kV)
N° Barra -> N° Barra
PA (MW) -> PA (MW)
CUST (MW) -> PA (MW)
Ano -> Ano
Margem (MW) -> PA (MW)
FatorLimitante -> FatorLimitante
Observacao -> Observacao
Nome_Planilha SINDAT 

<ipython-input-2-775497238eae>:790: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df_final.groupby('Categoria Margem').size().reset_index(name='Contagem').to_excel(


Renomeando colunas para compatibilidade com shapefile:
  Ponto de Conexão -> Ponto_de_C
  Tensão (kV) -> Tensão_(kV
  Margem (MW) -> Margem_(MW
  FatorLimitante -> FatorLimit
  Nome_Planilha SINDAT ONS -> Nome_Plani
  Coordenada X -> Coordenada
  Coordenada Y -> Coordena_1
  Categoria Margem -> Categoria_
  Categoria Tensão -> Categori_1
Shapefile criado com sucesso em: resultados/shapefile/pontos_capacidade.shp
  adding: resultados/shapefile/ (stored 0%)
  adding: resultados/shapefile/pontos_capacidade.dbf (deflated 97%)
  adding: resultados/shapefile/pontos_capacidade.shx (deflated 73%)
  adding: resultados/shapefile/pontos_capacidade.shp (deflated 82%)
  adding: resultados/shapefile/pontos_capacidade.cpg (stored 0%)
  adding: resultados/shapefile/pontos_capacidade.prj (deflated 17%)


...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


Processamento completo! Obrigado por utilizar o script.
